In [1]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('SEC_lit_releases_2012.csv')

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df.head()

,Release No.,Date,Action
0,LR-22582,"Dec. 27, 2012",Rajat K. Gupta and Raj Rajaratnam
1,LR-22581,"Dec. 26, 2012","Thomas C. Conradt, et al. See also: SEC Complaint"
2,LR-22580,"Dec. 21, 2012","GLR Capital Management, LLC, GLR Advisors, LLC, John A. Geringer, and Relief Defendant GLR Growth Fund, L.P."
3,LR-22579,"Dec. 21, 2012","Danny Garber, Michael Manis, Kenneth Yellin, Jordan Feinstein, Aluma Holdings LLC, Azure Trading LLC, Coastal Group Holdings, Inc., Greyhawk Equities LLC, Leonidas Group Holdings LLC, The Leonidas Group LLC, Nismic Sales Corp., The OGP Group, Perlinda Enterprises LLC, Rio Sterling Holding LLC, Slow Train Holding LLC, and Spartan Group Holdings LLC See also: SEC Complaint"
4,LR-22578,"Dec. 21, 2012",Lee S. Rosen See also: SEC Complaint


In [5]:
# make a new column of lowercase release numbers to generate URL column
df['release_no'] = df['Release No.'].str.replace('-', '')
df['release_no'] = df['release_no'].str.lower()

In [6]:
# add the url column based on the url pattern
df['url'] = "https://www.sec.gov/litigation/litreleases/2012/" + df.release_no + ".htm"

In [7]:
# drop the release_no column
df = df.drop(['release_no'], axis=1)

In [8]:
df

,Release No.,Date,Action,url
0,LR-22582,"Dec. 27, 2012",Rajat K. Gupta and Raj Rajaratnam,https://www.sec.gov/litigation/litreleases/2012/lr22582.htm
1,LR-22581,"Dec. 26, 2012","Thomas C. Conradt, et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2012/lr22581.htm
2,LR-22580,"Dec. 21, 2012","GLR Capital Management, LLC, GLR Advisors, LLC, John A. Geringer, and Relief Defendant GLR Growth Fund, L.P.",https://www.sec.gov/litigation/litreleases/2012/lr22580.htm
3,LR-22579,"Dec. 21, 2012","Danny Garber, Michael Manis, Kenneth Yellin, Jordan Feinstein, Aluma Holdings LLC, Azure Trading LLC, Coastal Group Holdings, Inc., Greyhawk Equities LLC, Leonidas Group Holdings LLC, The Leonidas Group LLC, Nismic Sales Corp., The OGP Group, Perlinda Enterprises LLC, Rio Sterling Holding LLC, Slow Train Holding LLC, and Spartan Group Holdings LLC See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2012/lr22579.htm
4,LR-22578,"Dec. 21, 2012",Lee S. Rosen See also: SEC Complaint,https://www.sec.gov/litigation/litreleases/2012/lr22578.htm
5,LR-22577,"Dec. 20, 2012","Rex T. Shelby, Scott Yeager, Kevin A. Howard and Michael W. Krautz; Schuyler M. Tilney and Thomas W. Davis Other Release No.: AAER-3434",https://www.sec.gov/litigation/litreleases/2012/lr22577.htm
6,LR-22576,"Dec. 20, 2012",Eli Lilly and Company See also: SEC Complaint,https://www.sec.gov/litigation/litreleases/2012/lr22576.htm
7,LR-22575,"Dec. 18, 2012","TheStreet, Inc., Eric Ashma, and Gregg Alwine and David Barnett See also: SEC Complaint - TheStreet, Inc. and SEC Complaint - Eric Ashma",https://www.sec.gov/litigation/litreleases/2012/lr22575.htm
8,LR-22574,"Dec. 17, 2012","Spencer Pharmaceutical Inc., Jean-François Amyot, Maximilien Arella, Ian Morrice, IAB Media Inc., and Hilbroy Advisory Inc. See also: Trading Suspension; SEC Complaint",https://www.sec.gov/litigation/litreleases/2012/lr22574.htm
9,LR-22573,"Dec. 14, 2012","Aletheia Research and Management, Inc. and Peter J. Eichler, Jr. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2012/lr22573.htm


In [9]:
# using BeautifulSoup to loop through each URL and pull all the text

def get_release_info(row):
    
    try:
        request = requests.get(row['url'])
        soup = BeautifulSoup(request.text, 'html.parser')
        
        full_text = soup.findAll(text=True)
        
    except:
        return pd.Series({})

    return pd.Series({
        'full_text': full_text,
    })

In [10]:
df_litreleases_2012 = df.apply(get_release_info, axis=1).join(df)

In [11]:
df_litreleases_2012.head()

full_text  \
0  [HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN", 
, 
, 
, Rajat K. Gupta and Raj Rajaratnam (Release No. LR-22582; December 27, 2012), 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, Home,  | , Previous Page, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, U.S. SECURITIES AND EXCHANGE COMMISSION, 
, Litigation Release No. 22582 / December 27, 2012, 
, Securities and Exchange Commission v. Rajat K. Gupta and Raj Rajaratnam, , Civil Action No. 11-CV-7566 (SDNY) (JSR), 
,  <h2>headline</h2> , 
,  Text goes here , 
, SEC Obtains Final Judgment on Consent as to Raj Rajaratnam, 
, Washington, D.C., December 27, 2012,  — The Securities and Exchange Commission today announced that, on December 26, 2012, the Honorable Jed S. Rakoff, United States District Judge, United States District Court for the Southern District of New York, entered a Final Judgment, on consent, as to former hedge fund manager Raj Rajaratnam in the SEC’s insider trading case, , SEC v. Rajat K. Gupta and Raj Rajaratnam,,  Civil Action No. 11-CV-7566 (SDNY) (JSR).  The final judgment orders Rajaratnam to pay $1,299,120 in disgorgement and $147,738, in prejudgment interest, for a total of $1,446,858.  , 
, The SEC’s complaint, filed October 26, 2011, alleges that, among other things, Rajat K. Gupta tipped his business associate Raj Rajaratnam, Galleon Management’s founder and managing general partner, to confidential information Gupta learned in the course of his duties as a member of the Board of Directors of The Goldman Sachs Group, Inc.  The complaint alleges that Gupta disclosed material nonpublic information concerning Berkshire Hathaway Inc.’s $5 billion investment in Goldman Sachs in September 2008, and concerning Goldman Sachs’s financial results for both the second and the fourth quarter of 2008.  Rajaratnam used the information he learned from Gupta to trade profitably in certain Galleon hedge funds.  By engaging in this conduct, Gupta and Rajaratnam violated Section 10(b) of the Securities Exchange Act of 1934, Exchange Act Rule 10b-5, and Section 17(a) of the Securities Act of 1933.  , 
, On June 15, 2012, in a parallel criminal case arising out of the same facts, Gupta was convicted of one count of conspiracy to commit securities fraud and three counts of securities fraud.  On October 24, 2012, Gupta was sentenced to two years in prison and one year of supervised release, and ordered to pay a $5 million criminal fine.  , 
, The Final Judgment in the SEC’s case orders Rajaratnam to disgorge his share of the profits gained and losses avoided as a result of the insider trading plus prejudgment interest on that amount.  The SEC’s claims against Gupta remain pending., 
,  End text , 
, Complaint  , 
,  <p><img src="/images/arrowright_dkblue.gif" width=10 height=9 alt="" border="0">
        <a href="/litigation/complaints/xxxx/compxxxxx.pdf">SEC Complaint</a></p>, 
,  , 
, http://www.sec.gov/litigation/litreleases/2012/lr22582.htm, 
, 
, 
, 
, Home,  | , Previous Page, 
, Modified: 12/27/2012, ...]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [12]:
df_litreleases_2012.dtypes

full_text      object
Release No.    object
Date           object
Action         object
url            object
dtype: object

In [13]:
df_litreleases_2012['Respondents'] = df_litreleases_2012['Action']

In [14]:
# rearrange columns
df_litreleases_2012 = df_litreleases_2012[['Release No.','Respondents','Date','url', 'full_text']]
df_litreleases_2012.head()

Release No.  \
0  LR-22582     
1  LR-22581     
2  LR-22580     
3  LR-22579     
4  LR-22578     

                                                                                                                                                                                                                                                                                                                                                                             Respondents  \
0  Rajat K. Gupta and Raj Rajaratnam                                                                                                                                                                                                                                                                                                                                                       
1  Thomas C. Conradt, et al. See also: SEC Complaint                                                                                                                                                                                                                                                                                                                                       
2  GLR Capital Management, LLC, GLR Advisors, LLC, John A. Geringer, and Relief Defendant GLR Growth Fund, L.P.                                                                                                                                                                                                                                                                            
3  Danny Garber, Michael Manis, Kenneth Yellin, Jordan Feinstein, Aluma Holdings LLC, Azure Trading LLC, Coastal Group Holdings, Inc., Greyhawk Equities LLC, Leonidas Group Holdings LLC, The Leonidas Group LLC, Nismic Sales Corp., The OGP Group, Perlinda Enterprises LLC, Rio Sterling Holding LLC, Slow Train Holding LLC, and Spartan Group Holdings LLC See also: SEC Complaint   
4  Lee S. Rosen See also: SEC Complaint                                                                                                                                                                                                                                                                                                                                                    

            Date                                                          url  \
0  Dec. 27, 2012  https://www.sec.gov/litigation/litreleases/2012/lr22582.htm   
1  Dec. 26, 2012  https://www.sec.gov/litigation/litreleases/2012/lr22581.htm   
2  Dec. 21, 2012  https://www.sec.gov/litigation/litreleases/2012/lr22580.htm   
3  Dec. 21, 2012  https://www.sec.gov/litigation/litreleases/2012/lr22579.htm   
4  Dec. 21, 2012  https://www.sec.gov/litigation/litreleases/2012/lr22578.htm   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [15]:
# Save to CSV
df_litreleases_2012.to_csv("litreleases_2012_scraped.csv", index=False)